### **Dim Customers**

In [0]:
%sql
CREATE OR REPLACE TABLE datamodeling.gold.DimCustomers
AS
WITH rem_dup AS
(
SELECT 
  DISTINCT(customer_id),
  customer_email,
  customer_name,
  Customer_Name_Upper
FROM datamodeling.silver.silver_table
)
SELECT * ,
      row_number() OVER (ORDER BY customer_id) as DimCustomerKey
FROM rem_dup

num_affected_rows,num_inserted_rows


### **Dim Products**

In [0]:
%sql
CREATE OR REPLACE TABLE datamodeling.gold.DimProducts
AS
WITH rem_dup AS
(
SELECT 
  DISTINCT(product_id),
  product_name,
  product_category
FROM 
  datamodeling.silver.silver_table
)
SELECT * ,
      row_number() OVER (ORDER BY product_id) as DimProductKey
FROM rem_dup

num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from datamodeling.gold.dimproducts

product_id,product_name,product_category,DimProductKey
501,iPhone 14,Electronics,1
502,AirPods Pro,Electronics,2
503,Nike Shoes,Footwear,3
504,Samsung S23,Electronics,4


### **Dim Payments**

In [0]:
%sql
CREATE OR REPLACE TABLE datamodeling.gold.DimPayments
WITH rem_dup AS 
(
SELECT 
  DISTINCT(payment_type)
FROM datamodeling.silver.silver_table
) 
SELECT *, row_number() OVER (ORDER BY payment_type) as DimPaymentKey FROM rem_dup

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM datamodeling.gold.DimPayments

payment_type,DimPaymentKey
Credit Card,1
PayPal,2


### **Dim Region**

In [0]:
%sql
CREATE OR REPLACE TABLE datamodeling.gold.DimRegions
WITH rem_dup AS 
(
SELECT 
  DISTINCT(country)
FROM datamodeling.silver.silver_table
) 
SELECT *, row_number() OVER (ORDER BY country) as DimRegionKey FROM rem_dup

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM datamodeling.gold.dimregions

country,DimRegionKey
Canada,1
USA,2


### **Dim Sales**

In [0]:
%sql
CREATE OR REPLACE TABLE datamodeling.gold.DimSales
AS
SELECT 
 row_number() OVER (ORDER BY order_id) as DimSaleKey,
 order_id,
 order_date,
 customer_id,
 customer_name,
 customer_email,
 product_id,
 product_name,
 product_category,
 payment_type,
 country,
 last_updated,
 Customer_Name_Upper,
 processDate
FROM
  datamodeling.silver.silver_table

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM datamodeling.gold.dimsales

DimSaleKey,order_id,order_date,customer_id,customer_name,customer_email,product_id,product_name,product_category,payment_type,country,last_updated,Customer_Name_Upper,processDate
1,1001,2024-07-01,1,Alice Johnson,alice@gmail.com,501,iPhone 14,Electronics,Credit Card,USA,2024-07-01,ALICE JOHNSON,2025-07-12
2,1002,2024-07-01,2,Bob Smith,bob@yahoo.com,502,AirPods Pro,Electronics,PayPal,USA,2024-07-01,BOB SMITH,2025-07-12
3,1003,2024-07-01,3,Charlie Brown,charlie@outlook.com,503,Nike Shoes,Footwear,Credit Card,Canada,2024-07-01,CHARLIE BROWN,2025-07-12
4,1004,2024-07-02,4,David Lee,david@abc.com,504,Samsung S23,Electronics,Credit Card,USA,2024-07-02,DAVID LEE,2025-07-12
5,1005,2024-07-02,1,Alice Johnson,alice@gmail.com,503,Nike Shoes,Footwear,Credit Card,USA,2024-07-02,ALICE JOHNSON,2025-07-12


### **FACT TABLE**

In [0]:
%sql
CREATE OR REPLACE TABLE datamodeling.gold.FactSales
AS
SELECT 
  S.DimSaleKey,
  C.DimCustomerKey,
  P.DimProductKey,
  R.DimRegionKey,
  PY.DimPaymentKey,
  F.quantity,
  F.unit_price
FROM 
  datamodeling.silver.silver_table F
LEFT JOIN 
  datamodeling.gold.dimcustomers C
  ON F.customer_id = C.customer_id
LEFT JOIN 
  datamodeling.gold.dimproducts P
  ON F.product_id = P.product_id
LEFT JOIN 
  datamodeling.gold.dimregions R
  ON F.country = R.country
LEFT JOIN 
  datamodeling.gold.dimpayments PY
  ON F.payment_type = PY.payment_type
LEFT JOIN 
  datamodeling.gold.dimsales S
  ON F.order_id = S.order_id

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM datamodeling.gold.FactSales

DimSaleKey,DimCustomerKey,DimProductKey,DimRegionKey,DimPaymentKey,quantity,unit_price
1,1,1,2,1,1,999.99
2,2,2,2,2,2,199.99
3,3,3,1,1,1,129.99
4,4,4,2,1,1,899.99
5,1,3,2,1,2,129.99
